    affine_transform.txt is in Ground Truth folders.
    
    steps:
    - find the subject ids 
    - find the corresponding affine_transform.txt file for the id in each project.
    - use the affine_transform.txt to transform the subject predictions.
        - note: the subject predictions could be .stl, .ply format.

In [2]:
import os
import numpy as np
import trimesh
from tqdm import tqdm
import pickle

In [3]:
with open('subjects.pkl', 'rb') as f:
    subjects = pickle.load(f)

In [4]:
dic = {}

dic['corticalflow'] = {
    "lh_pial": "/data/users2/llu13/output/corticalflow/cfpp/lh_pial",
    "lh_white": "/data/users2/llu13/output/corticalflow/cfpp/lh_white",
}

dic['deepcsr'] = {
    "lh_pial": "/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial",
    "lh_white": "/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_white",
}


dic['vox2cortex'] = {
    "lh_pial": "/data/users2/washbee/speedrun/Vox2Cortex_fork/experiments/hcp/test_template_42016_DATASET_NAME/meshes/lh_pial/epoch81",
    "lh_white": "/data/users2/washbee/speedrun/Vox2Cortex_fork/experiments/hcp/test_template_42016_DATASET_NAME/meshes/lh_white/epoch81",
}

dic_stl = dic

In [5]:
c,d,v = 'corticalflow', 'deepcsr', 'vox2cortex'

In [6]:
# folders for transform_affine.txt, (inside ground truth folders)

dic_ta = {
    c: "/data/users2/washbee/speedrun/deepcsr-preprocessed/",
    d: "/data/users2/washbee/speedrun/deepcsr-preprocessed/",
    v: "/data/users2/washbee/speedrun/Vox2Cortex_fork/CSR_data"
}

# -------------examples-------------
# "/data/users2/washbee/speedrun/deepcsr-preprocessed/201818/transform_affine.txt"
# "/data/users2/washbee/speedrun/Vox2Cortex_fork/CSR_data/201818/transform_affine.txt"

In [7]:
f = os.path.join(dic_stl[v]['lh_pial'], os.listdir(dic_stl[v]['lh_pial'])[0])
f = '/data/users2/washbee/speedrun/Vox2Cortex_fork/experiments/hcp/test_template_42016_DATASET_NAME/meshes/lh_pial/epoch81/200008_epoch81_struc2_meshpred.ply'

In [8]:
def transform(mesh_file, affine_file, save_path):
    mesh = trimesh.load_mesh(mesh_file)
    affine_transform = np.loadtxt(affine_file)
    mesh.apply_transform(affine_transform)
    mesh.export(save_path)

In [10]:
for i in [-4,-3,-2,-1]:
    dic2 = {}
    subject = subjects[i]
    dic2 = {
    c: [f'{subject}_lh_pial_Df2.pial_converted.stl', 
        f'{subject}_lh_white_Df2.white_converted.stl', 
        f'/data/users2/washbee/speedrun/deepcsr-preprocessed/{subject}/transform_affine.txt'],
    d: [f'{subject}_lh_pial.stl', 
        f'{subject}_lh_white.stl', 
        f'/data/users2/washbee/speedrun/deepcsr-preprocessed/{subject}/transform_affine.txt'],
    v: [f'{subject}_epoch81_struc2_meshpred.ply', 
        f'{subject}_epoch81_struc0_meshpred.ply',
        f"/data/users2/washbee/speedrun/Vox2Cortex_fork/CSR_data/{subject}/transform_affine.txt"]
    }
    for k in dic2:
        model = k
        pial_file, white_file, affine_file = dic2[model]
        
        pial_file_path = os.path.join(dic_stl[model]['lh_pial'], pial_file)
        white_file_path = os.path.join(dic_stl[model]['lh_white'], white_file)
        
        pial_save_path = f'{subject}-{model}-transformed-pial.stl'
        white_save_path = f'{subject}-{model}-transformed-white.stl'
       
        try:
            transform(pial_file_path, affine_file, pial_save_path)
        except:
            print(k, pial_file_path)
        
        try:
            transform(white_file_path, affine_file, white_save_path)
        except:
            print(k, white_file_path)